In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import image_loader
import torch
csv_file = '../data/movie_data2.csv'
img_dir = '../data/MoviePosters'
pod = image_loader.PosterDataset(csv_file, img_dir, genres = None)

In [3]:
from torch.utils.data import DataLoader

trainSize = int(len(pod) * 0.9)
testSize = len(pod) - trainSize

trainData, testData = torch.utils.data.random_split(pod, [trainSize, testSize])
def batch_function(raw_output):
    xs, ys = zip(*raw_output)
    batchxs = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)
    batchys = torch.Tensor(ys)
    return batchxs, batchys

BATCH_SIZE = 5
trainDataLoader = DataLoader(trainData, batch_size = BATCH_SIZE, shuffle=True, collate_fn=batch_function)
testDataLoader = DataLoader(testData, batch_size = 5, shuffle=True)



In [4]:
xs, ys = next(iter(trainDataLoader))
xs.shape

torch.Size([5, 3, 256, 256])

In [5]:
len(trainData)

31995

In [6]:
# download VGG19 model
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# vgg_model = models.vgg19_bn(pretrained=True, progress=True).features.to(device).eval()

In [8]:
# check model architecture
# print(model)

In [22]:
# Initialize the model
# num_features = model.features[-1].out_channels  # Get the number of output channels of the last convolutional layer
# model.features.add_module('7', nn.Conv2d(num_features, 1, 1))  # Add a 1x1 convolutional layer to change the output size to 1
# model = model.to(device)

# Define the loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:
class MoviePredictor(torch.nn.Module):
    def __init__(self, vggmodel, linear_params):
        super().__init__()
        self.predictor_1 = torch.nn.Linear(512 * 8 * 8, linear_params)
        self.out = torch.nn.Linear(linear_params, 1)
    def forward(self, encoded: torch.Tensor):
        """
        encoded: Batch x (512 * 8 * 8)
        return: Batch x 1
        """
        output = self.out(self.predictor_1(encoded))
        return output

In [61]:
class DeepDropPredictor(nn.Module):
    def __init__(self, dropout_p, num_extra_lin, hidden_dim):
        super().__init__()
        self.flattened = nn.Linear(512 * 8 * 8, hidden_dim)
        self.lin_layers = nn.ParameterList([nn.Linear(hidden_dim, hidden_dim) for _ in range(num_extra_lin)])
        self.nonlin = nn.LeakyReLU()
        self.out = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(p=dropout_p)
    def forward(self, encoded: torch.Tensor):
        """
        encoded: Batch x (512 * 8 * 8)
        retrun: Batch x 1
        """
        encoded = self.flattened(encoded)
        for layer in self.lin_layers:
            encoded = self.dropout(self.nonlin(layer(encoded)))
        output = self.out(encoded)
        # output = 10 * torch.sigmoid(output)
        return output

In [9]:
class FullPosterPredictor(nn.Module):
    def __init__(self, encoder, predictor) -> None:
        super().__init__()
        self.encoder = encoder
        self.predictor = predictor
    def forward(self, images):
        encodings = self.encoder(images)
        predictions = self.predictor(encodings)
        return predictions

In [62]:
import AutoencoderModels as AEM

vgg_model = models.vgg19_bn(weights=None).features
empty_encoder = AEM.FlatVGG(vgg_model)
empty_decoder = AEM.PosterDecoder()
aenc = AEM.PosterAutoEncoder(empty_encoder, empty_decoder)

aenc.load_state_dict(torch.load("../models/vgg_arch_ae.pt"))

encoder = aenc.encoder.to(device).requires_grad_(False)
predictor = DeepDropPredictor(0.5, 0, 256).to(device)
poster_predictor = FullPosterPredictor(encoder, predictor).to(device)

predictor_loss = nn.MSELoss(reduction="mean")
predictor_optim = torch.optim.Adam(predictor.parameters(), lr=1e-3)

In [63]:
poster_predictor(pod[222][0][None,:,:,:].to(device))
# pod[222][1]
# n = 6
# plt.imshow(torch.unflatten(encoder(pod[222][0][None,:,:,:].to(device)).squeeze(), -1, (512, 8, 8)).cpu().mean(axis=0, keepdim=True).permute((1, 2, 0)))

tensor([[0.1433]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [14]:
# Train the model
from tqdm import tqdm
import numpy as np

def train_model(in_model, criterion, optimizer, num_epochs):
    # uses global training data
    print(f"Training for {num_epochs} epoch(s).")
    for epoch in range(num_epochs):
        print("------------------------")
        in_model.train()
        running_loss = 0.0
        pbar = tqdm(trainDataLoader)
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = in_model(images)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_description(f"Loss: {loss.item():.5f}")

        print(f'Epoch {epoch+1}, Average Train Loss: {running_loss/len(trainDataLoader):.5f}')

        # Test the model
        print("Validating Model")
        in_model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for images, labels in tqdm(testDataLoader):
                images, labels = images.to(device), labels.to(device)
                outputs = in_model(images)
                loss = criterion(outputs.squeeze(), labels.float().squeeze())
                test_loss += loss.item()
        print(f'Validation Loss: {test_loss/len(testDataLoader)}')

In [52]:
next(iter(trainDataLoader))[1]

tensor([6.1000, 4.0000, 3.9000, 7.4000, 6.9000])

In [16]:
device

device(type='cuda')

In [64]:
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 5.30813: 100%|██████████| 6399/6399 [06:22<00:00, 16.71it/s] 


Epoch 1, Average Train Loss: 13.52551
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.16it/s]

Validation Loss: 1.5004594215465665


In [65]:
predictor_optim = torch.optim.Adam(predictor.parameters(), lr=1e-4)
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 0.57053: 100%|██████████| 6399/6399 [06:26<00:00, 16.55it/s]


Epoch 1, Average Train Loss: 1.34435
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.14it/s]

Validation Loss: 1.3602153981411005


In [66]:
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 1.52720: 100%|██████████| 6399/6399 [06:26<00:00, 16.54it/s]


Epoch 1, Average Train Loss: 1.33444
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.15it/s]

Validation Loss: 1.2732233402074304


In [67]:
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 0.41296: 100%|██████████| 6399/6399 [06:29<00:00, 16.44it/s]


Epoch 1, Average Train Loss: 1.32166
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.17it/s]

Validation Loss: 1.4310167590588456


In [68]:
predictor_optim = torch.optim.Adam(predictor.parameters(), lr=1e-5)
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 0.60132: 100%|██████████| 6399/6399 [06:25<00:00, 16.59it/s]


Epoch 1, Average Train Loss: 1.24856
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.22it/s]

Validation Loss: 1.2474873604674714


In [69]:
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 0.45909: 100%|██████████| 6399/6399 [06:34<00:00, 16.23it/s]


Epoch 1, Average Train Loss: 1.24899
Validating Model


100%|██████████| 711/711 [00:41<00:00, 17.16it/s]

Validation Loss: 1.2353745406931043


In [70]:
predictor_optim = torch.optim.Adam(predictor.parameters(), lr=1e-6)
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 0.28731: 100%|██████████| 6399/6399 [06:43<00:00, 15.86it/s]


Epoch 1, Average Train Loss: 1.24007
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.26it/s]

Validation Loss: 1.2353461495536457


In [71]:
torch.save(poster_predictor.state_dict(), "../models/ae_predictor_small.pt")

In [73]:
large_predictor = DeepDropPredictor(0.5, 3, 512).to(device)
large_poster_predictor = FullPosterPredictor(encoder, large_predictor).to(device)
large_predictor_optim = torch.optim.Adam(large_predictor.parameters(), lr=1e-3)

train_model(large_poster_predictor, predictor_loss, large_predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 3.93249: 100%|██████████| 6399/6399 [06:52<00:00, 15.50it/s]    


Epoch 1, Average Train Loss: 42.16783
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.19it/s]

Validation Loss: 1.7000384305914242


In [74]:
large_predictor_optim = torch.optim.Adam(large_predictor.parameters(), lr=1e-4)

train_model(large_poster_predictor, predictor_loss, large_predictor_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 2.00278: 100%|██████████| 6399/6399 [06:53<00:00, 15.48it/s] 


Epoch 1, Average Train Loss: 2.40324
Validating Model


100%|██████████| 711/711 [00:39<00:00, 17.99it/s]


Validation Loss: 21.543933803108214
------------------------


Loss: 2.52884: 100%|██████████| 6399/6399 [07:55<00:00, 13.46it/s]


Epoch 2, Average Train Loss: 1.70637
Validating Model


100%|██████████| 711/711 [00:51<00:00, 13.93it/s]

Validation Loss: 1.515928999323047


In [75]:
train_model(large_poster_predictor, predictor_loss, large_predictor_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 1.89076: 100%|██████████| 6399/6399 [06:53<00:00, 15.49it/s] 


Epoch 1, Average Train Loss: 1.63040
Validating Model


100%|██████████| 711/711 [00:39<00:00, 17.98it/s]


Validation Loss: 2.026799619491258
------------------------


Loss: 1.66885: 100%|██████████| 6399/6399 [07:23<00:00, 14.42it/s] 


Epoch 2, Average Train Loss: 1.58885
Validating Model


100%|██████████| 711/711 [00:45<00:00, 15.63it/s]

Validation Loss: 1.707868715609344


In [76]:
large_predictor_optim = torch.optim.Adam(large_predictor.parameters(), lr=1e-5)

train_model(large_poster_predictor, predictor_loss, large_predictor_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 1.32570: 100%|██████████| 6399/6399 [06:53<00:00, 15.48it/s]


Epoch 1, Average Train Loss: 1.49412
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.17it/s]


Validation Loss: 1.4292399885916216
------------------------


Loss: 2.12397: 100%|██████████| 6399/6399 [06:53<00:00, 15.47it/s]


Epoch 2, Average Train Loss: 1.47972
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.00it/s]

Validation Loss: 1.5062006573741613


In [77]:
train_model(large_poster_predictor, predictor_loss, large_predictor_optim, 3)

Training for 3 epoch(s).
------------------------


Loss: 2.67664: 100%|██████████| 6399/6399 [07:09<00:00, 14.90it/s] 


Epoch 1, Average Train Loss: 1.46708
Validating Model


100%|██████████| 711/711 [00:41<00:00, 17.25it/s]


Validation Loss: 1.493059268009059
------------------------


...

Validation Loss: 1.586419178673319
------------------------


Loss: 1.78983: 100%|██████████| 6399/6399 [07:09<00:00, 14.91it/s]


Epoch 3, Average Train Loss: 1.46067
Validating Model


100%|██████████| 711/711 [00:40<00:00, 17.40it/s]


Validation Loss: 1.652398806353494


In [78]:
large_predictor_optim = torch.optim.Adam(large_predictor.parameters(), lr=1e-6)

train_model(large_poster_predictor, predictor_loss, large_predictor_optim, 3)

Training for 3 epoch(s).
------------------------


Loss: 4.60054: 100%|██████████| 6399/6399 [06:47<00:00, 15.69it/s]


Epoch 1, Average Train Loss: 1.44292
Validating Model


100%|██████████| 711/711 [00:39<00:00, 17.92it/s]


Validation Loss: 1.6046348018296661
------------------------


...

Validation Loss: 1.5308945899260782
------------------------


Loss: 1.68257: 100%|██████████| 6399/6399 [06:59<00:00, 15.24it/s]


Epoch 3, Average Train Loss: 1.45106
Validating Model


100%|██████████| 711/711 [00:39<00:00, 17.99it/s]

Validation Loss: 1.5827514007739965


In [79]:
large_predictor_optim = torch.optim.Adam(large_predictor.parameters(), lr=1e-7)

train_model(large_poster_predictor, predictor_loss, large_predictor_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 0.79274: 100%|██████████| 6399/6399 [06:48<00:00, 15.67it/s]


Epoch 1, Average Train Loss: 1.44524
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.17it/s]


Validation Loss: 1.5567234089249633
------------------------


Loss: 1.10852: 100%|██████████| 6399/6399 [06:54<00:00, 15.43it/s] 


Epoch 2, Average Train Loss: 1.43318
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.13it/s]

Validation Loss: 1.5764311486763458


In [80]:
torch.save(large_poster_predictor.state_dict(), "../models/ae_predictor_large.pt")

In [81]:
med_predictor = DeepDropPredictor(0.5, 2, 128).to(device)
med_poster_predictor = FullPosterPredictor(encoder, med_predictor).to(device)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-3)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 1)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-4)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 2)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-5)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 3)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-6)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 4)

Training for 1 epoch(s).
------------------------


Loss: 3.14296: 100%|██████████| 6399/6399 [06:18<00:00, 16.92it/s]  


Epoch 1, Average Train Loss: 3.11870
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.19it/s]


Validation Loss: 1.4996366329732156
Training for 2 epoch(s).
------------------------


...

Validation Loss: 1.498754837761113
------------------------


Loss: 0.85546: 100%|██████████| 6399/6399 [06:17<00:00, 16.94it/s] 


Epoch 4, Average Train Loss: 1.63274
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.45it/s]

Validation Loss: 1.53327212214428


In [82]:
torch.save(med_poster_predictor.state_dict(), "../models/ae_predictor_med.pt")

In [83]:
# should have been med_small for all this
med_predictor = DeepDropPredictor(0.5, 2, 256).to(device)
med_poster_predictor = FullPosterPredictor(encoder, med_predictor).to(device)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-3)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 1)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-4)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 2)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-5)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 3)

med_predictor_optim = torch.optim.Adam(med_predictor.parameters(), lr=1e-6)
train_model(med_poster_predictor, predictor_loss, med_predictor_optim, 4)

Training for 1 epoch(s).
------------------------


Loss: 3.47485: 100%|██████████| 6399/6399 [06:54<00:00, 15.43it/s]   


Epoch 1, Average Train Loss: 5.87325
Validating Model


100%|██████████| 711/711 [00:42<00:00, 16.78it/s]


Validation Loss: 1.8668353737825751
Training for 2 epoch(s).
------------------------


...

Validation Loss: 1.4579265676491204
------------------------


Loss: 1.38077: 100%|██████████| 6399/6399 [06:34<00:00, 16.24it/s]


Epoch 4, Average Train Loss: 1.49395
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.21it/s]

Validation Loss: 1.4846960312769384


In [84]:
torch.save(med_poster_predictor.state_dict(), "../models/ae_predictor_small_med.pt")

In [85]:
m_1_256_predictor = DeepDropPredictor(0.5, 1, 256).to(device)
m_1_256_poster_predictor  = FullPosterPredictor(encoder, m_1_256_predictor).to(device)

m_1_256_optim = torch.optim.Adam(m_1_256_predictor.parameters(), lr=1e-3)
train_model(m_1_256_poster_predictor, predictor_loss, m_1_256_optim, 1)

m_1_256_optim = torch.optim.Adam(m_1_256_predictor.parameters(), lr=1e-4)
train_model(m_1_256_poster_predictor, predictor_loss, m_1_256_optim, 2)

m_1_256_optim = torch.optim.Adam(m_1_256_predictor.parameters(), lr=1e-5)
train_model(m_1_256_poster_predictor, predictor_loss, m_1_256_optim, 3)

m_1_256_optim = torch.optim.Adam(m_1_256_predictor.parameters(), lr=1e-6)
train_model(m_1_256_poster_predictor, predictor_loss, m_1_256_optim, 4)

torch.save(m_1_256_poster_predictor.state_dict(), "../models/ae_predictor_1_256.pt")


Training for 1 epoch(s).
------------------------


Loss: 1.56463: 100%|██████████| 6399/6399 [06:26<00:00, 16.57it/s] 


Epoch 1, Average Train Loss: 2.74780
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.20it/s]


Validation Loss: 1.2845478188534663
Training for 2 epoch(s).
------------------------


...

Validation Loss: 1.237815044946057
------------------------


Loss: 1.27508: 100%|██████████| 6399/6399 [06:25<00:00, 16.59it/s]


Epoch 4, Average Train Loss: 1.38470
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.41it/s]


Validation Loss: 1.2379347553369282


In [86]:
m_1_512_predictor = DeepDropPredictor(0.5, 1, 512).to(device)
m_1_512_poster_predictor  = FullPosterPredictor(encoder, m_1_512_predictor).to(device)

m_1_512_optim = torch.optim.Adam(m_1_512_predictor.parameters(), lr=1e-3)
train_model(m_1_512_poster_predictor, predictor_loss, m_1_512_optim, 1)

m_1_512_optim = torch.optim.Adam(m_1_512_predictor.parameters(), lr=1e-4)
train_model(m_1_512_poster_predictor, predictor_loss, m_1_512_optim, 2)

m_1_512_optim = torch.optim.Adam(m_1_512_predictor.parameters(), lr=1e-5)
train_model(m_1_512_poster_predictor, predictor_loss, m_1_512_optim, 3)

m_1_512_optim = torch.optim.Adam(m_1_512_predictor.parameters(), lr=1e-6)
train_model(m_1_512_poster_predictor, predictor_loss, m_1_512_optim, 4)


torch.save(m_1_512_poster_predictor.state_dict(), "../models/ae_predictor_1_512.pt")

Training for 1 epoch(s).
------------------------


Loss: 0.53918: 100%|██████████| 6399/6399 [06:46<00:00, 15.73it/s] 


Epoch 1, Average Train Loss: 3.45567
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.09it/s]


Validation Loss: 2.2901584600031626
Training for 2 epoch(s).
------------------------


...

Validation Loss: 1.2354700567067172
------------------------


Loss: 1.61800: 100%|██████████| 6399/6399 [06:45<00:00, 15.78it/s]


Epoch 4, Average Train Loss: 1.32821
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.37it/s]


Validation Loss: 1.235795244380685


In [87]:
train_model(m_1_512_poster_predictor, predictor_loss, m_1_512_optim, 3)
torch.save(m_1_512_poster_predictor.state_dict(), "../models/ae_predictor_1_512.pt")

Training for 3 epoch(s).
------------------------


Loss: 1.44336: 100%|██████████| 6399/6399 [06:56<00:00, 15.37it/s]


Epoch 1, Average Train Loss: 1.33022
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.01it/s]


Validation Loss: 1.2394272151747627
------------------------


...

Validation Loss: 1.2361961644399602
------------------------


Loss: 1.78162: 100%|██████████| 6399/6399 [06:51<00:00, 15.56it/s]


Epoch 3, Average Train Loss: 1.31902
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.18it/s]


Validation Loss: 1.2422366310303976


In [88]:
m_1_768_predictor = DeepDropPredictor(0.5, 1, 768).to(device)
m_1_768_poster_predictor  = FullPosterPredictor(encoder, m_1_768_predictor).to(device)

m_1_768_optim = torch.optim.Adam(m_1_768_predictor.parameters(), lr=1e-3)
train_model(m_1_768_poster_predictor, predictor_loss, m_1_768_optim, 1)

m_1_768_optim = torch.optim.Adam(m_1_768_predictor.parameters(), lr=1e-4)
train_model(m_1_768_poster_predictor, predictor_loss, m_1_768_optim, 2)

m_1_768_optim = torch.optim.Adam(m_1_768_predictor.parameters(), lr=1e-5)
train_model(m_1_768_poster_predictor, predictor_loss, m_1_768_optim, 2)

m_1_768_optim = torch.optim.Adam(m_1_768_predictor.parameters(), lr=1e-6)
train_model(m_1_768_poster_predictor, predictor_loss, m_1_768_optim, 3)

torch.save(m_1_768_poster_predictor.state_dict(), "../models/ae_predictor_1_768.pt")

Training for 1 epoch(s).
------------------------


Loss: 4.64174: 100%|██████████| 6399/6399 [07:17<00:00, 14.62it/s] 


Epoch 1, Average Train Loss: 5.02483
Validating Model


100%|██████████| 711/711 [00:41<00:00, 16.93it/s]


Validation Loss: 1.7774738790113715
Training for 2 epoch(s).
------------------------


...

Validation Loss: 1.2424404287577178
------------------------


Loss: 0.87158: 100%|██████████| 6399/6399 [07:11<00:00, 14.84it/s]


Epoch 3, Average Train Loss: 1.31537
Validating Model


100%|██████████| 711/711 [00:39<00:00, 17.87it/s]


Validation Loss: 1.2408191255711638


In [89]:
m_3_128_predictor = DeepDropPredictor(0.5, 3, 128).to(device)
m_3_128_poster_predictor  = FullPosterPredictor(encoder, m_3_128_predictor).to(device)

m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-3)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 1)

m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-4)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 2)

m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-5)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 2)

m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-6)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 3)

torch.save(m_3_128_poster_predictor.state_dict(), "../models/ae_predictor_3_128.pt")

Training for 1 epoch(s).
------------------------


Loss: 2.96664: 100%|██████████| 6399/6399 [06:46<00:00, 15.72it/s] 


Epoch 1, Average Train Loss: 3.85858
Validating Model


100%|██████████| 711/711 [00:43<00:00, 16.46it/s]


Validation Loss: 1.6585110361817517
Training for 2 epoch(s).
------------------------


...

Validation Loss: 2.1439602483383546
------------------------


Loss: 2.69787: 100%|██████████| 6399/6399 [07:02<00:00, 15.14it/s]


Epoch 3, Average Train Loss: 1.74694
Validating Model


100%|██████████| 711/711 [00:42<00:00, 16.69it/s]


Validation Loss: 2.090725616207438


In [90]:
m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-3)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 0.20271: 100%|██████████| 6399/6399 [06:19<00:00, 16.87it/s]    


Epoch 1, Average Train Loss: 4.28058
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.18it/s]


Validation Loss: 1.2709101166487076
------------------------


Loss: 3.15630: 100%|██████████| 6399/6399 [06:24<00:00, 16.63it/s] 


Epoch 2, Average Train Loss: 1.65030
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.19it/s]

Validation Loss: 1.2677590017286868


In [91]:
m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-4)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 1.86355: 100%|██████████| 6399/6399 [06:24<00:00, 16.64it/s]


Epoch 1, Average Train Loss: 1.44486
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.31it/s]


Validation Loss: 1.2547398444458988
------------------------


Loss: 1.52950: 100%|██████████| 6399/6399 [06:24<00:00, 16.63it/s]


Epoch 2, Average Train Loss: 1.40803
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.30it/s]

Validation Loss: 1.2490925170984197


In [92]:
m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-5)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 1.36689: 100%|██████████| 6399/6399 [06:22<00:00, 16.71it/s]


Epoch 1, Average Train Loss: 1.38543
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.23it/s]


Validation Loss: 1.2707366450007287
------------------------


Loss: 1.59350: 100%|██████████| 6399/6399 [06:25<00:00, 16.61it/s]


Epoch 2, Average Train Loss: 1.37878
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.28it/s]

Validation Loss: 1.2624457269624065


In [93]:
m_3_128_optim = torch.optim.Adam(m_3_128_predictor.parameters(), lr=1e-4)
train_model(m_3_128_poster_predictor, predictor_loss, m_3_128_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 2.17243: 100%|██████████| 6399/6399 [06:30<00:00, 16.39it/s] 


Epoch 1, Average Train Loss: 1.38965
Validating Model


100%|██████████| 711/711 [00:43<00:00, 16.45it/s]

Validation Loss: 1.2637433328111716


In [94]:
torch.save(m_3_128_poster_predictor.state_dict(), "../models/ae_predictor_3_128.pt")

In [95]:
m_5_256_predictor = DeepDropPredictor(0.5, 5, 256).to(device)
m_5_256_poster_predictor  = FullPosterPredictor(encoder, m_5_256_predictor).to(device)

m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-3)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 3)

m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-4)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 3)

m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-5)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 4)

m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-6)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 5)

torch.save(m_5_256_poster_predictor.state_dict(), "../models/ae_predictor_5_256.pt")

Training for 3 epoch(s).
------------------------


Loss: 1.19560: 100%|██████████| 6399/6399 [06:31<00:00, 16.33it/s] 


Epoch 1, Average Train Loss: 4.71152
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.18it/s]


Validation Loss: 2.5179683412890226
------------------------


...

Validation Loss: 1.3163162902123315
------------------------


Loss: 1.44055: 100%|██████████| 6399/6399 [06:31<00:00, 16.35it/s]      


Epoch 5, Average Train Loss: 344.32405
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.45it/s]


Validation Loss: 1.3106891449865703


In [97]:
m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-4)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 3)


Training for 3 epoch(s).
------------------------


Loss: 0.98437: 100%|██████████| 6399/6399 [06:32<00:00, 16.30it/s]   


Epoch 1, Average Train Loss: 1.65494
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.39it/s]


Validation Loss: 1.3272432743580869
------------------------


...

Validation Loss: 1.3301844892436547
------------------------


Loss: 0.52392: 100%|██████████| 6399/6399 [06:36<00:00, 16.15it/s] 


Epoch 3, Average Train Loss: 1.32162
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.33it/s]

Validation Loss: 1.3252044059964927


In [98]:
m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-5)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 3)


Training for 3 epoch(s).
------------------------


Loss: 4.08348: 100%|██████████| 6399/6399 [06:40<00:00, 15.97it/s]


Epoch 1, Average Train Loss: 1.31199
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.03it/s]


Validation Loss: 1.2983026903033592
------------------------


...

Validation Loss: 1.289642442216895
------------------------


Loss: 2.41348: 100%|██████████| 6399/6399 [06:37<00:00, 16.08it/s] 


Epoch 3, Average Train Loss: 1.31204
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.22it/s]

Validation Loss: 1.3024542929680623


In [99]:
m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-5)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 2)


Training for 2 epoch(s).
------------------------


Loss: 1.78437: 100%|██████████| 6399/6399 [06:32<00:00, 16.32it/s]   


Epoch 1, Average Train Loss: 2.65210
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.14it/s]


Validation Loss: 1.2925207373776517
------------------------


Loss: 0.61734: 100%|██████████| 6399/6399 [06:41<00:00, 15.92it/s]


Epoch 2, Average Train Loss: 1.30259
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.19it/s]

Validation Loss: 1.2908000625387024


In [100]:
m_5_256_optim = torch.optim.Adam(m_5_256_predictor.parameters(), lr=1e-5)
train_model(m_5_256_poster_predictor, predictor_loss, m_5_256_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 0.73978: 100%|██████████| 6399/6399 [06:49<00:00, 15.64it/s] 


Epoch 1, Average Train Loss: 1.30382
Validating Model


100%|██████████| 711/711 [00:39<00:00, 18.06it/s]

Validation Loss: 1.28070092839405


In [101]:
torch.save(m_5_256_poster_predictor.state_dict(), "../models/ae_predictor_5_256.pt")

In [102]:
m_0_1024_predictor = DeepDropPredictor(0.5, 0, 1024).to(device)
m_0_1024_poster_predictor  = FullPosterPredictor(encoder, m_0_1024_predictor).to(device)

m_0_1024_optim = torch.optim.Adam(m_0_1024_predictor.parameters(), lr=1e-3)
train_model(m_0_1024_poster_predictor, predictor_loss, m_0_1024_optim, 1)

m_0_1024_optim = torch.optim.Adam(m_0_1024_predictor.parameters(), lr=1e-4)
train_model(m_0_1024_poster_predictor, predictor_loss, m_0_1024_optim, 2)

Training for 1 epoch(s).
------------------------


Loss: 3.23469: 100%|██████████| 6399/6399 [07:57<00:00, 13.41it/s]  


Epoch 1, Average Train Loss: 36.83900
Validating Model


100%|██████████| 711/711 [00:42<00:00, 16.71it/s]


Validation Loss: 1.3411047882367706
Training for 2 epoch(s).
------------------------


...

Validation Loss: 1.2866790610746743
------------------------


Loss: 0.53237: 100%|██████████| 6399/6399 [07:46<00:00, 13.73it/s]


Epoch 2, Average Train Loss: 1.39090
Validating Model


100%|██████████| 711/711 [00:41<00:00, 17.06it/s]

Validation Loss: 1.2882121247285256


In [103]:
m_0_1024_optim = torch.optim.Adam(m_0_1024_predictor.parameters(), lr=1e-5)
train_model(m_0_1024_poster_predictor, predictor_loss, m_0_1024_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 0.28581: 100%|██████████| 6399/6399 [07:52<00:00, 13.54it/s]


Epoch 1, Average Train Loss: 1.26103
Validating Model


100%|██████████| 711/711 [00:38<00:00, 18.28it/s]


Validation Loss: 1.2628529025543005
------------------------


Loss: 0.28737: 100%|██████████| 6399/6399 [07:53<00:00, 13.52it/s]


Epoch 2, Average Train Loss: 1.25782
Validating Model


100%|██████████| 711/711 [00:43<00:00, 16.24it/s]

Validation Loss: 1.241142852596006


In [104]:
m_0_1024_optim = torch.optim.Adam(m_0_1024_predictor.parameters(), lr=1e-5)
train_model(m_0_1024_poster_predictor, predictor_loss, m_0_1024_optim, 2)

Training for 2 epoch(s).
------------------------


Loss: 1.25978: 100%|██████████| 6399/6399 [07:53<00:00, 13.52it/s]


Epoch 1, Average Train Loss: 1.25547
Validating Model


100%|██████████| 711/711 [00:43<00:00, 16.43it/s]


Validation Loss: 1.2408780940924935
------------------------


Loss: 0.14188: 100%|██████████| 6399/6399 [08:05<00:00, 13.19it/s]


Epoch 2, Average Train Loss: 1.25337
Validating Model


100%|██████████| 711/711 [00:41<00:00, 17.10it/s]

Validation Loss: 1.2425586701214901


In [105]:
m_0_1024_optim = torch.optim.Adam(m_0_1024_predictor.parameters(), lr=1e-6)
train_model(m_0_1024_poster_predictor, predictor_loss, m_0_1024_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 2.29424: 100%|██████████| 6399/6399 [07:35<00:00, 14.04it/s]


Epoch 1, Average Train Loss: 1.24206
Validating Model


100%|██████████| 711/711 [00:40<00:00, 17.55it/s]

Validation Loss: 1.23615256009516


In [106]:
m_0_1024_optim = torch.optim.Adam(m_0_1024_predictor.parameters(), lr=1e-6)
train_model(m_0_1024_poster_predictor, predictor_loss, m_0_1024_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 1.23035: 100%|██████████| 6399/6399 [07:31<00:00, 14.17it/s]


Epoch 1, Average Train Loss: 1.24064
Validating Model


100%|██████████| 711/711 [00:39<00:00, 17.99it/s]

Validation Loss: 1.236184659994269


In [107]:
torch.save(m_0_1024_poster_predictor.state_dict(), "../models/ae_predictor_0_1024.pt")

In [96]:
torch.save(trainData, "../datasets/aencpredTrainData.dataset")
torch.save(testData, "../datasets/aencpredTestData.dataset")

In [58]:
list(predictor.out.parameters())[-2].numel()

256

In [18]:
predictor_optim = optim.Adam(predictor.parameters(), lr=1e-2)
train_model(poster_predictor, predictor_loss, predictor_optim, 1)

Training for 1 epoch(s).
------------------------


Loss: 1052633728.00000:   1%|          | 61/6399 [00:04<07:53, 13.40it/s]


KeyboardInterrupt: 

In [41]:

torch.save(trainData, "../datasets/traindata.dataset")
torch.save(testData, "../datasets/testdata.dataset")

In [42]:
for x in tqdm(trainDataLoader):
    pass

100%|██████████| 6399/6399 [01:21<00:00, 78.95it/s]


In [52]:
deepDropPredictor = DeepDropPredictor(model, 0.5, 2, 512).to(device)
deepDropPredictor.vgg_model.requires_grad_(False)
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-4)

train_model(deepDropPredictor, loss_fn, deepOptim, 3)

Training for 3 epoch(s).


Loss: 1.8596395254135132: 100%|██████████| 6399/6399 [06:49<00:00, 15.64it/s] 


Epoch 1, Average Loss: 2.9606533144912293


Loss: 3.578465223312378: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]   


Epoch 2, Average Loss: 2.2760654106849123


Loss: 0.6861000657081604: 100%|██████████| 6399/6399 [06:50<00:00, 15.59it/s]  


Epoch 3, Average Loss: 2.156359913838545


100%|██████████| 3555/3555 [00:48<00:00, 73.69it/s]

Test Loss: 2.220100910033067


In [53]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-5)

train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 3.026665210723877: 100%|██████████| 6399/6399 [06:50<00:00, 15.59it/s]   


Epoch 1, Average Loss: 1.753013373617619


Loss: 1.1039469242095947: 100%|██████████| 6399/6399 [06:51<00:00, 15.55it/s]  


Epoch 2, Average Loss: 1.6522094191074288


...

Epoch 4, Average Loss: 1.5690152994923936


Loss: 0.7266332507133484: 100%|██████████| 6399/6399 [07:04<00:00, 15.07it/s]  


Epoch 5, Average Loss: 1.5444395596618194


100%|██████████| 3555/3555 [00:51<00:00, 68.55it/s]


Test Loss: 1.8330523182734366


In [58]:
def test_model(in_model, criterion):
    in_model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(testDataLoader):
            images, labels = images.to(device), labels.to(device)
            outputs = in_model(images)
            loss = criterion(outputs.squeeze(), labels.float().squeeze())
            test_loss += loss.item()
    print(f'Test Loss: {test_loss/len(testDataLoader)}')

def test_empty_model(in_model, criterion):
    # in_model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(testDataLoader):
            images, labels = images.to(device), labels.to(device)
            outputs = in_model(images)
            loss = criterion(outputs.squeeze(), labels.float().squeeze())
            test_loss += loss.item()
    print(f'Test Loss: {test_loss/len(testDataLoader)}')

In [59]:
test_empty_model(lambda x: torch.Tensor([6.413]).to(device), loss_fn) # average score

100%|██████████| 3555/3555 [00:15<00:00, 222.46it/s]

Test Loss: 1.2836419191611301


In [60]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-6)
train_model(deepDropPredictor, loss_fn, deepOptim, 2)

Training for 2 epoch(s).


Loss: 1.1587564945220947: 100%|██████████| 6399/6399 [07:02<00:00, 15.16it/s]  


Epoch 1, Average Loss: 1.5048829187346664


Loss: 1.2700437307357788: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]  


Epoch 2, Average Loss: 1.4929980461582177


100%|██████████| 3555/3555 [00:47<00:00, 74.83it/s]

Test Loss: 1.4232098539967803


In [61]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-5)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 2.1003499031066895: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]  


Epoch 1, Average Loss: 1.5296387848145776


Loss: 1.0658739805221558: 100%|██████████| 6399/6399 [06:53<00:00, 15.46it/s]  


Epoch 2, Average Loss: 1.515018851160514


...

Epoch 4, Average Loss: 1.5061488871211293


Loss: 2.510054111480713: 100%|██████████| 6399/6399 [07:10<00:00, 14.87it/s]   


Epoch 5, Average Loss: 1.4934773749917405


100%|██████████| 3555/3555 [00:50<00:00, 70.23it/s]

Test Loss: 1.5648263567632892


In [62]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-4)
train_model(deepDropPredictor, loss_fn, deepOptim, 1)

Training for 1 epoch(s).


Loss: 0.34157657623291016: 100%|██████████| 6399/6399 [06:52<00:00, 15.52it/s] 


Epoch 1, Average Loss: 2.021788834831569


100%|██████████| 3555/3555 [00:47<00:00, 74.66it/s]

Test Loss: 1.9029515225770162


In [63]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-5)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 0.7813587188720703: 100%|██████████| 6399/6399 [06:48<00:00, 15.66it/s]  


Epoch 1, Average Loss: 1.6962225304342775


Loss: 1.3287068605422974: 100%|██████████| 6399/6399 [06:51<00:00, 15.56it/s]  


Epoch 2, Average Loss: 1.5597085176024992


...

Epoch 4, Average Loss: 1.4975741139133822


Loss: 1.428186058998108: 100%|██████████| 6399/6399 [06:51<00:00, 15.55it/s]   


Epoch 5, Average Loss: 1.4949296167531347


100%|██████████| 3555/3555 [00:47<00:00, 74.71it/s]

Test Loss: 1.604877375682443


In [65]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-6)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 4.137628555297852: 100%|██████████| 6399/6399 [06:46<00:00, 15.73it/s]   


Epoch 1, Average Loss: 1.455965


Loss: 1.1817193031311035: 100%|██████████| 6399/6399 [06:50<00:00, 15.59it/s]  


Epoch 2, Average Loss: 1.451000


...

Epoch 4, Average Loss: 1.440130


Loss: 1.4763487577438354: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]  


Epoch 5, Average Loss: 1.439623
Testing Model


100%|██████████| 3555/3555 [00:47<00:00, 74.62it/s]

Test Loss: 1.5110017234956894


In [68]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-7)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 2.065505: 100%|██████████| 6399/6399 [06:49<00:00, 15.64it/s]


Epoch 1, Average Loss: 1.426922


Loss: 1.551556: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]


Epoch 2, Average Loss: 1.420094


...

Epoch 4, Average Loss: 1.426952


Loss: 0.416120: 100%|██████████| 6399/6399 [06:46<00:00, 15.73it/s] 


Epoch 5, Average Loss: 1.415997
Testing Model


100%|██████████| 3555/3555 [00:46<00:00, 76.76it/s]

Test Loss: 1.4303539512436128


In [69]:
# just keep going why not
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-7)
train_model(deepDropPredictor, loss_fn, deepOptim, 3)

Training for 3 epoch(s).


Loss: 0.686216: 100%|██████████| 6399/6399 [06:50<00:00, 15.60it/s]


Epoch 1, Average Loss: 1.420392


Loss: 1.540256: 100%|██████████| 6399/6399 [06:51<00:00, 15.54it/s]


Epoch 2, Average Loss: 1.425687


Loss: 1.138829: 100%|██████████| 6399/6399 [06:51<00:00, 15.57it/s]


Epoch 3, Average Loss: 1.427274
Testing Model


100%|██████████| 3555/3555 [00:48<00:00, 73.10it/s]

Test Loss: 1.7407562702686945


In [70]:
torch.save(deepDropPredictor.state_dict(), "../models/DeepDrop3-512.pt")

In [80]:
total_params = 0
for param in list(deepDropPredictor.parameters())[-8:]:
    print(param.size().numel())
    total_params += param.size().numel()
print(f"{total_params:,}")

16777216
512
262144
512
262144
512
512
1
17,303,553
